In [3]:
import pandas as pd
#pd.set_option('max_rows', 10)
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import statsmodels.api as statsmodels # useful stats package with regression functions
import seaborn as sns #plotting package
import zipfile

In [16]:
# Importing the data from https://datacatalog.worldbank.org/search/dataset/0037712/World-Development-Indicators
# ONLY RUN ONCE

# from urllib.request import urlopen
# from io import BytesIO
# from zipfile import ZipFile
# 
# def download_and_unzip(url, extract_to='.'):
#      print("Downloading")
#      http_response = urlopen(url)
#      zipfile = ZipFile(BytesIO(http_response.read()))
#      print("Unziping")
#      zipfile.extractall(path=extract_to)
#      print("Done")
# 
# download_and_unzip("https://databank.worldbank.org/data/download/WDI_csv.zip",pathname+"/Data/Raw")


In [4]:
pathname = "/Users/norika_machome/GitHub/Norika_DS4SG-Global-Inequality"

wdi_data = pd.read_csv(pathname + "/Data/Raw/WDIData.csv", delimiter=",")
wdi_codebook = pd.read_csv(pathname+"/Data/Raw/WDISeries.csv", delimiter=",")

In [46]:
merged = pd.merge(wdi_data, wdi_codebook, how='inner',
                   left_on='Indicator Name', right_on='Indicator Name')
merged['Topic'].nunique() # 89
wdi_data.head()

#Checking which indicators don't have a defined Topic to see if it's okay to omit them for our study
# df1 = merged[merged['Topic'].isnull()]
# display(set(df1["Indicator Name"]))

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Unnamed: 66
0,Africa Eastern and Southern,AFE,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,16.936004,17.337896,17.687093,18.140971,18.491344,18.825520,19.272212,19.628009,NaN,NaN
1,Africa Eastern and Southern,AFE,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.RU.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,6.499471,6.680066,6.859110,7.016238,7.180364,7.322294,7.517191,7.651598,NaN,NaN
2,Africa Eastern and Southern,AFE,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.UR.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,37.855399,38.046781,38.326255,38.468426,38.670044,38.722783,38.927016,39.042839,NaN,NaN
3,Africa Eastern and Southern,AFE,Access to electricity (% of population),EG.ELC.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,31.794160,32.001027,33.871910,38.880173,40.261358,43.061877,44.270860,45.803485,NaN,NaN
4,Africa Eastern and Southern,AFE,"Access to electricity, rural (% of rural popul...",EG.ELC.ACCS.RU.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,18.663502,17.633986,16.464681,24.531436,25.345111,27.449908,29.641760,30.404935,NaN,NaN


In [21]:
#showing all the available columns ???
#pd.set_option('max_columns', None) ???
#print(merged.columns.tolist()) ???

Creating Separate datasets for each type of indicator (facilitates understanding and reduces size for upload to Github)

In [23]:
def create_df_topic(topics, write_to = None):
    # Select only indicators from topics
    topic_df = merged.loc[merged["Topic"].isin(topics)]
    # Melt dataset (making all date columns one variable)
    long_topic_df = topic_df.melt(id_vars=["Country Name", "Indicator Name",'Country Code', 'Topic', 'Indicator Code'], 
        var_name="Date")
    # Write to csv
    if write_to:
        long_topic_df.to_csv(pathname + "/Data/Final/" + write_to + ".csv", index = False)
        print("Written to:", pathname + "/Data/Final/" + write_to + ".csv")
    return long_topic_df


In [34]:
def create_df_indicator(indicator_code, write_to = None):
    # Select only indicators from topics
    topic_df = merged.loc[merged["Indicator Code"].isin(indicator_code)]
    # Melt dataset (making all date columns one variable)
    long_indicator_df = topic_df.melt(id_vars=["Country Name",'Country Code', 'Topic', 'Indicator Code',"Indicator Name"], 
        var_name="Date")
    # Write to csv
    if write_to:
        long_indicator_df.to_csv(pathname + "/Data/Final/" + write_to + ".csv", index = False)
        print("Written to:", pathname + "/Data/Final/" + write_to + ".csv")
    return long_indicator_df


In [44]:
test_df = create_df_indicator(["SL.UEM.NEET.ZS","SE.SEC.TCAQ.ZS", "SP.DYN.LE00.IN", "SP.REG.BRTH.FE.ZS","SI.DST.FRST.10",'SI.DST.10TH.10',"NY.GDP.PCAP.CD"],"test")
test_df = test_df[test_df['value'].notna()]
test_df



Written to: /Users/norika_machome/GitHub/Norika_DS4SG-Global-Inequality/Data/Final/test.csv


,Country Name,Country Code,Topic,Indicator Code,Indicator Name,Date,value
266,Africa Eastern and Southern,AFE,Economic Policy & Debt: National accounts: US$...,NY.GDP.PCAP.CD,GDP per capita (current US$),1960,162.726325
267,Africa Western and Central,AFW,Economic Policy & Debt: National accounts: US$...,NY.GDP.PCAP.CD,GDP per capita (current US$),1960,107.930722
269,Caribbean small states,CSS,Economic Policy & Debt: National accounts: US$...,NY.GDP.PCAP.CD,GDP per capita (current US$),1960,448.256417
271,Early-demographic dividend,EAR,Economic Policy & Debt: National accounts: US$...,NY.GDP.PCAP.CD,GDP per capita (current US$),1960,161.165351
272,East Asia & Pacific,EAS,Economic Policy & Debt: National accounts: US$...,NY.GDP.PCAP.CD,GDP per capita (current US$),1960,148.423623
...,...,...,...,...,...,...,...
150817,Virgin Islands (U.S.),VIR,Education: Inputs,SE.SEC.TCAQ.ZS,Trained teachers in secondary education (% of ...,License Type,CC BY-4.0
150818,West Bank and Gaza,PSE,Education: Inputs,SE.SEC.TCAQ.ZS,Trained teachers in secondary education (% of ...,License Type,CC BY-4.0
150819,"Yemen, Rep.",YEM,Education: Inputs,SE.SEC.TCAQ.ZS,Trained teachers in secondary education (% of ...,License Type,CC BY-4.0
150820,Zambia,ZMB,Education: Inputs,SE.SEC.TCAQ.ZS,Trained teachers in secondary education (% of ...,License Type,CC BY-4.0


In [ ]:
#education_df = create_df_topic(['Education: Outcomes','Education: Participation'], "Education")
#poverty_df = create_df_topic(['Poverty: Income distribution','Poverty: Poverty rates'], "Poverty")
#mortality_df = create_df_topic(['Health: Mortality'], "Mortality")
#labor_df = create_df_topic(["Social Protection & Labor: Unemployment", "Social Protection & Labor: Labor force structure"], "Labor")
#environmnet_df = create_df_topic(["Environment: Land use", "Environment: Emissions"], "Environment")

In [ ]:
#checking out all of the indicators
#print(set(education_df["Indicator Name"]))
#education_df.describe()
#checking to see if any of the rows have non-NA values
#selected_rows = long_df[~long_df['value'].isnull()]
#print(selected_rows)
len(education_df)

In [137]:
def random_question_generator(df, indicator_name, n=4, year = "2018"):
    df = df[df["Indicator Name"] == indicator_name]
    df = df[df['value'].notna()]
    df_year = df[df['Date'] == year]
    df_year = df_year.sample(n = n)
    df_year = df_year.reset_index()
    #display(df_year)

    for i in range(4):
        country_name = df_year["Country Name"][i]
        country_value = df_year["value"][i]
        print("Index Name:",indicator_name, "Country: ", country_name, "Value: ", country_value)

random_question_generator(education_df, 'Adjusted net enrollment rate, primary (% of primary school age children)', year = "2018")




Index Name: Adjusted net enrollment rate, primary (% of primary school age children) Country:  Middle East & North Africa (excluding high income) Value:  94.73726
Index Name: Adjusted net enrollment rate, primary (% of primary school age children) Country:  Sri Lanka Value:  99.27348
Index Name: Adjusted net enrollment rate, primary (% of primary school age children) Country:  Moldova Value:  89.78655
Index Name: Adjusted net enrollment rate, primary (% of primary school age children) Country:  Low income Value:  80.48845


In [117]:
random_question_generator(enrollment_rate)


,index,Country Name,Indicator Name,Country Code,Topic,Indicator Code,Date,value
0,1698286,Ireland,"Adjusted net enrollment rate, primary (% of pr...",IRL,Education: Participation,SE.PRM.TENR,Development relevance,Relevance to gender indicator: Women teachers ...
1,1117215,Heavily indebted poor countries (HIPC),"Adjusted net enrollment rate, primary (% of pr...",HPC,Education: Participation,SE.PRM.TENR,2010,76.23378
2,1407903,St. Lucia,"Adjusted net enrollment rate, primary (% of pr...",LCA,Education: Participation,SE.PRM.TENR,Series Code,SE.PRM.TENR
3,1586558,Honduras,"Adjusted net enrollment rate, primary (% of pr...",HND,Education: Participation,SE.PRM.TENR,Limitations and exceptions,Enrollment indicators are based on annual scho...


country:  Ireland value:  Relevance to gender indicator: Women teachers are important as they serve as role models to girls and help to attract and retain girls in school.
country:  Heavily indebted poor countries (HIPC) value:  76.23378
country:  St. Lucia value:  SE.PRM.TENR
country:  Honduras value:  Enrollment indicators are based on annual school surveys, but do not necessarily reflect actual attendance or dropout rates during the year. Also, the length of education differs across countries and can influence enrollment rates, although the International Standard Classification of Education (ISCED) tries to minimize the difference. For example, a shorter duration for primary education tends to increase the rate; a longer one to decrease it (in part because older children are more at risk of dropping out). Moreover, age at enrollment may be inaccurately estimated or misstated, especially in communities where registration of births is not strictly enforced.
